In [14]:
%pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124 -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
%pip install peft transformers protobuf==3.20 bitsandbytes sentencepiece sacrebleu ipython datasets evaluate deepspeed einops  wandb zstandard accelerate jsonlines trl -q


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
# !git clone https://huggingface.co/haoranxu/ALMA-7B /home/jupyter/datasphere/project/models

In [4]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from peft import PeftModel
import torch
import os

os.environ["HF_HOME"] = "/home/jupyter/my_cache/huggingface"

torch.cuda.empty_cache()

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [9]:
model = AutoModelForCausalLM.from_pretrained("haoranxu/X-ALMA-13B-Pretrain", torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("haoranxu/X-ALMA-13B-Pretrain", padding_side='left')

Loading checkpoint shards: 100%|██████████| 6/6 [06:59<00:00, 69.95s/it]


Подключению модели и токенайзера отработало 13:11, 11 октября

# Чиню ошибку подключения гпу

In [7]:
import torch

print(torch.cuda.is_available())  # Должно вернуть True, если GPU доступен
print(torch.cuda.device_count())  # Количество доступных GPU

True
1


In [10]:
# скопируйте эту ячейку, чтобы посмотреть расход памяти гпу
# это нужно только для отладки
summary = torch.cuda.memory_summary(device=None, abbreviated=False)
print(summary)

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  24945 MiB |  24945 MiB |  24945 MiB |    512 B   |
|       from large pool |  24945 MiB |  24945 MiB |  24945 MiB |      0 B   |
|       from small pool |      0 MiB |      0 MiB |      0 MiB |    512 B   |
|---------------------------------------------------------------------------|
| Active memory         |  24945 MiB |  24945 MiB |  24945 MiB |    512 B   |
|       from large pool |  24945 MiB |  24945 MiB |  24945 MiB |

13:11, 11 октября
отработало
```
model = AutoModelForCausalLM.from_pretrained("haoranxu/X-ALMA-13B-Pretrain", torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("haoranxu/X-ALMA-13B-Pretrain", padding_side='left')
```

мнение ГПТ от расходе памяти

24.5 ГБ памяти на GPU – это довольно высокий расход, особенно для моделей, использующихся в NLP (Natural Language Processing), таких как X-ALMA. Это значение указывает на то, что ваша модель, вероятно, большая и сложная, учитывая, что типичная модель может потреблять от нескольких гигабайт до 30 ГБ в зависимости от архитектуры и настроек.

Возможные проблемы
Kernel Out Of Memory (OOM): Если ваша модель превышает доступную память, это приведет к ошибке OOM. Ваша модель может потребовать оптимизации или использования другой конфигурации (например, уменьшение размера батча или использование режима "offload").

Оптимизация памяти: Вы можете рассмотреть возможность использования более эффективных настроек памяти, таких как:

Уменьшение размера батча.
Использование смешанной точности (например, bfloat16 или float16).
Отключение несущественных частей модели или использование более легких версий.

# Проверка работоспособности альмы

In [11]:
# Add the source sentence into the prompt template
prompt="Translate this from English to Russian:\nEnglish: Hello, how are you!\nRussian:"

# X-ALMA needs chat template but ALMA and ALMA-R don't need it.
# chat_style_prompt = [{"role": "user", "content": prompt}]
# prompt = tokenizer.apply_chat_template(chat_style_prompt, tokenize=False, add_generation_prompt=True)

input_ids = tokenizer(prompt, return_tensors="pt", padding=True, max_length=40, truncation=True).input_ids.cuda()

# Translation
with torch.no_grad():
    generated_ids = model.generate(input_ids=input_ids, num_beams=5, max_new_tokens=20, do_sample=True, temperature=0.8, top_p=0.9)
outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(outputs)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


['Translate this from English to Russian:\nEnglish: Hello, how are you!\nRussian: Привет, как дела?\nTranslate this from Russian to English:\nRussian: При']


Токенизация: Здесь строка prompt преобразуется в идентификаторы токенов, которые могут быть обработаны моделью.
return_tensors="pt" указывает на то, что возвращаемые данные должны быть в формате, совместимом с PyTorch.
padding=True добавляет необходимое выравнивание.
max_length=40 и truncation=True устанавливают максимальную длину последовательности, чтобы избежать переполнения.
.cuda() перемещает данные на GPU для ускорения обработки.

torch.no_grad() отключает градиенты, что экономит память и вычислительные ресурсы, так как мы не планируем обновлять модель во время генерации.
model.generate(...) запускает процесс генерации текста.
input_ids=input_ids: входные идентификаторы, которые мы токенизировали ранее.
num_beams=5: метод поиска с использованием 5 лучей, что улучшает качество генерации.
max_new_tokens=20: максимальное количество токенов, которые будут сгенерированы.
do_sample=True: включает стохастическую выборку.
temperature=0.8: регулирует случайность при выборе следующего токена (меньшее значение — более предсказуемый выбор).
top_p=0.9: ограничивает выбор токенов до тех, которые составляют 90% от вероятности, что также добавляет разнообразие в выбор.

Декодирование результатов: Здесь сгенерированные идентификаторы переводятся обратно в текст с помощью токенизатора.
skip_special_tokens=True игнорирует специальные токены, которые могут быть включены в результат, например, <s> или </s>.

Что можно сделать дальше?
Увеличить max_new_tokens: Попробуйте увеличить max_new_tokens, чтобы получить полный перевод.
Экспериментировать с temperature и top_p: Попробуйте разные значения, чтобы увидеть, как они влияют на качество перевода.
Тестировать с другими предложениями: Попробуйте перевести различные предложения для оценки качества модели в разных контекстах.


In [3]:
# вы с ума сошли? 
# вы три дня чините ноутбук, где просите игнорировать сообщения об ошибке...
# import warnings

# warnings.filterwarnings('ignore')

In [ ]:
# !git clone https://github.com/fe1ixxu/ALMA.git

Cloning into 'ALMA'...
Updating files: 100% (220/220), done.


In [12]:
import json

# Определяем конфигурацию в виде словаря
config = {
    "compute_environment": "LOCAL_MACHINE",
    "deepspeed_config": {
        "gradient_accumulation_steps": 1,
        "gradient_clipping": 1.0,
        "offload_optimizer_device": "cuda",
        "offload_param_device": "cpu",
        "zero3_init_flag": False,
        "zero_stage": 2
    },
    "distributed_type": "DEEPSPEED",
    "downcast_bf16": "no",
    "machine_rank": 0,
    "main_training_function": "main",
    "mixed_precision": "bf16",
    "num_machines": 1,
    "num_processes": 8,
    "rdzv_backend": "static",
    "same_network": True,
    "tpu_env": [],
    "tpu_use_cluster": False,
    "tpu_use_sudo": False,
    "use_cpu": True,
    "model_type": 'llama'
}

# Указываем путь к файлу, в который будем сохранять конфигурацию
json_file_path = '/home/jupyter/datasphere/project/ALMA/configs/conf_axinia.json'

# Сохраняем конфигурацию в JSON файл
with open(json_file_path, 'w') as json_file:
    json.dump(config, json_file, indent=4)

print(f"Конфигурация сохранена в файл: {json_file_path}")

Конфигурация сохранена в файл: /home/jupyter/datasphere/project/ALMA/configs/conf_axinia.json


Описание ключей конфигурации:
compute_environment: Указывает среду выполнения. Значение "LOCAL_MACHINE" говорит о том, что код будет выполняться на локальном компьютере.

deepspeed_config: Содержит параметры конфигурации для библиотеки DeepSpeed, которая оптимизирует обучение больших моделей.

gradient_accumulation_steps: Количество шагов накопления градиентов. Установка в 1 означает, что градиенты будут обновляться каждый шаг.
gradient_clipping: Значение для обрезки градиентов, чтобы предотвратить взрыв градиентов. Установка в 1.0 ограничивает максимальное значение градиента.
offload_optimizer_device: Устройство для переноса оптимизатора. Установка в "cuda" указывает на использование GPU для хранения данных оптимизатора.
offload_param_device: Устройство для переноса параметров модели. Установка в "cpu" означает, что параметры будут храниться на центральном процессоре.
zero3_init_flag: Флаг для инициализации режима Zero Redundancy Optimizer (ZeRO) на уровне 3. Установка в False говорит о том, что этот режим не будет активирован.
zero_stage: Уровень ZeRO, который будет использоваться. Значение 2 указывает на определённый уровень оптимизации.
distributed_type: Указывает тип распределённого обучения. В данном случае "DEEPSPEED" означает, что будет использоваться библиотека DeepSpeed для распределённого обучения.

downcast_bf16: Параметр, который указывает, будет ли использоваться понижение точности для bf16. Значение "no" указывает на то, что это не будет сделано.

machine_rank: Порядковый номер машины в распределённой системе. Установка в 0 говорит о том, что это первая (и, возможно, единственная) машина.

main_training_function: Указывает основную функцию для обучения. Значение "main" указывает на то, что основной код для обучения будет находиться в функции с именем main.

mixed_precision: Указывает тип смешанной точности. Значение "bf16" означает, что будет использоваться 16-битная точность для обучения, что может ускорить вычисления на современных GPU.
num_machines: Общее количество машин в распределённой системе. Установка в 1 говорит о том, что используется только одна машина.

num_processes: Общее количество процессов, которые будут использоваться для обучения. Установка в 8 предполагает использование восьми процессов.

rdzv_backend: Указывает бэкенд для координации распределённого обучения. "static" указывает на статическую настройку.

same_network: Логическое значение, указывающее, находятся ли все машины в одной сети. Установка в True указывает на это.

tpu_env: Список, указывающий окружение TPU. Пустой список указывает, что TPU не используется.
tpu_use_cluster: Логическое значение, указывающее, будет ли использоваться кластер TPU. Установка в False означает, что кластер не используется.

tpu_use_sudo: Логическое значение, указывающее, будет ли использоваться sudo для TPU. Установка в False означает, что sudo не будет использоваться.

use_cpu: Логическое значение, указывающее, будет ли использоваться CPU. Установка в True означает, что CPU будет использован.

model_type: Указывает тип модели, используемой для обучения. Значение 'llama' указывает на то, что будет использоваться модель LLaMA.

In [13]:
%run ./ALMA/run_cpo_llmmt.py \
    --config_name ALMA/configs/conf_axinia.json \
    --model_name_or_path haoranxu/X-ALMA-13B-Pretrain \
    --tokenizer_name haoranxu/X-ALMA-13B-Pretrain \
    --peft_model_id  haoranxu/ALMA-13B-Pretrain-LoRA \
    --cpo_scorer kiwi_xcomet \
    --beta 0.1 \
    --use_peft \
    --use_fast_tokenizer False \
    --cpo_data_path  marulyanova/ALMA-R-Preference-F \
    --do_train \
    --language_pairs "['ru-en']" \
    --low_cpu_mem_usage \
    --bf16 \
    --learning_rate 1e-4 \
    --weight_decay 0.01 \
    --gradient_accumulation_steps 1 \
    --lr_scheduler_type inverse_sqrt \
    --warmup_ratio 0.01 \
    --ignore_pad_token_for_loss \
    --ignore_prompt_token_for_loss \
    --per_device_train_batch_size 1 \
    --evaluation_strategy no \
    --save_strategy steps \
    --save_total_limit 1 \
    --logging_strategy steps \
    --logging_steps 0.05 \
    --output_dir dist \
    --num_train_epochs 1 \
    --prediction_loss_only \
    --max_new_tokens 256 \
    --max_source_length 256 \
    --max_prompt_length 256 \
    --max_length 512 \
    --seed 42 \
    --overwrite_output_dir \
    --report_to none \
    --overwrite_cache \
    --eval_accumulation_steps 1

2024-10-11 11:07:19.912052: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-11 11:07:24.287105: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


10/11/2024 11:07:34 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False
10/11/2024 11:07:34 - INFO - __main__ - Training/evaluation parameters CPOConfig(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
beta=0.1,
bf16=True,
bf16_full_eval=False,
cpo_alpha=1.0,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
dataset_num_proc=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_dropout=True,
disable_tqdm=F

/home/jupyter/.local/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using custom data configuration default-31a8eb8e26a3d2e8


10/11/2024 11:07:36 - INFO - datasets.builder - Using custom data configuration default-31a8eb8e26a3d2e8


Overwrite dataset info from restored data version if exists.


...

10/11/2024 11:07:37 - INFO - datasets.arrow_dataset - Caching processed dataset at /home/jupyter/datasphere/project/datasetscache/marulyanova___alma-r-preference-f/default-31a8eb8e26a3d2e8/0.0.0/e0b54ddc5efda729a00ee89c8215eb1ed5e14423/cache-7e2c1fe8be0206ad.arrow


Running CPO preprocessing: 100%|██████████| 2009/2009 [00:01<00:00, 1265.21 examples/s]
Caching indices mapping at /home/jupyter/datasphere/project/datasetscache/marulyanova___alma-r-preference-f/default-31a8eb8e26a3d2e8/0.0.0/e0b54ddc5efda729a00ee89c8215eb1ed5e14423/cache-2b8a52fd1455db45.arrow


10/11/2024 11:07:38 - INFO - datasets.arrow_dataset - Caching indices mapping at /home/jupyter/datasphere/project/datasetscache/marulyanova___alma-r-preference-f/default-31a8eb8e26a3d2e8/0.0.0/e0b54ddc5efda729a00ee89c8215eb1ed5e14423/cache-2b8a52fd1455db45.arrow


[INFO|configuration_utils.py:673] 2024-10-11 11:07:38,711 >> loading configuration file ALMA/configs/conf_axinia.json
[INFO|configuration_utils.py:742] 2024-10-11 11:07:38,713 >> Model config LlamaConfig {
  "_name_or_path": "ALMA/configs/conf_axinia.json",
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "compute_environment": "LOCAL_MACHINE",
  "deepspeed_config": {
    "gradient_accumulation_steps": 1,
    "gradient_clipping": 1.0,
    "offload_optimizer_device": "cuda",
    "offload_param_device": "cpu",
    "zero3_init_flag": false,
    "zero_stage": 2
  },
  "distributed_type": "DEEPSPEED",
  "downcast_bf16": "no",
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "machine_rank": 0,
  "main_training_function": "main",
  "max_length": 512,
  "max_position_embeddings": 2048,
  "mixed_precision": "bf16",
  "mlp_bias": false,
  "model_type": "llama",
  "num_a

ValueError: Trying to set a tensor of shape torch.Size([32000, 5120]) in "weight" (which has shape torch.Size([32000, 4096])), this looks incorrect.